In [53]:
# -*- coding: utf-8 -*-
import sys
import os
import json
from datetime import datetime
import numpy as np
import pandas as pd
import pymysql
import statsmodels.api as sm
import statsmodels.formula.api as smf
import predictions as pr
import finance as fr
import cost

params={'startdate':'2018-10-10','year':25,'size':99.5,'weight':1.5,'averagetime':3.4,
        "scene":"lm_model","type":"month","plant":"solar",
        "construction":150000000,"investment":180000000,"othercost":25000000,
        "principal":120000000,"equity":60000000,"interest":0.05, "unredeemed":12321321123,"duration":12,
        "repayment_method":"cpm","repayment_term":"m","interest_repayment_term":"q",
        "finance_startdate":"2018-06-13","price_index":0.01,"solar_index":-0.08}

#물가 예상 
price_index=pr.index_predict(params["startdate"],params["year"],params["price_index"]);price_index.name="price_index";
solar_index=pr.index_predict(params["startdate"],params["year"],params["solar_index"]);solar_index.name="solar_index";

#수입예상 및 전체적인 지표구성
revenue=pr.predict(params["scene"],params["startdate"],params["year"])
revenue["rec_price"]=100;
revenue["days"]=revenue.index.day
start=pd.to_datetime(params["startdate"], format='%Y-%m-%d', errors='ignore')#초기 기간 설정
startday=start.date().timetuple()
revenue.days[0]=revenue.days[0]-startday.tm_mday
revenue.days[revenue.shape[0]-1]=startday.tm_mday

revenue=pd.concat([revenue,price_index,solar_index],axis=1)
revenue["generation"]=revenue['days']*params['size']*params['averagetime']*revenue["solar_index"]
revenue['smp_revenue']=revenue['smp_price']*revenue['generation']
revenue['rec_revenue']=revenue['rec_price']*revenue['generation']*params['weight']
revenue["smp_revenue"]=revenue["smp_revenue"].astype(int); revenue["rec_revenue"]=revenue["rec_revenue"].astype(int)

#지출예상
OM_cost=cost.OM_calc(params["startdate"],params["size"],params["year"])
monitoring_cost=cost.monitoring_cost_calc(params["startdate"],params["size"],params["year"])
elec_safety_cost=cost.elec_safety_calc(params["startdate"],params["size"],params["year"])
office_cost=cost.office_cost_calc(params["startdate"],params["size"],params["year"])
other_cost=cost.other_cost_calc(params["startdate"],params["size"],params["year"])
total_cost=pd.concat([OM_cost,monitoring_cost,elec_safety_cost,office_cost,other_cost],axis=1)
cost=pd.concat([total_cost,price_index],axis=1)

cost["OM_cost"]=cost["OM_cost"]*cost["price_index"]
cost["monitoring_cost"]=cost["monitoring_cost"]*cost["price_index"]
cost["elec_safety_cost"]=cost["elec_safety_cost"]*cost["price_index"]
cost["office_cost"]=cost["office_cost"]*cost["price_index"]
cost["other_cost"]=cost["other_cost"]*cost["price_index"]
cost["depreciation"]=params["construction"]/(params["duration"]*12)

#금융구조 예상
if(params["repayment_method"]=="cam"):
    amortization=fr.CAM_calc(params["finance_startdate"],params["duration"],params["principal"],params["interest"],params["repayment_term"],params["interest_repayment_term"])
elif(params["repayment_method"]=="cpm"):
    amortization=fr.CPM_calc(params["finance_startdate"],params["duration"],params["principal"],params["interest"],params["repayment_term"])
amortization.index=amortization.index.shift(n=1,freq="m")
result=pd.concat([revenue,cost,amortization],axis=1)

#전체적인 구조 도출 
result=result.loc[:,['smp_revenue','rec_revenue','OM_cost','monitoring_cost','elec_safety_cost','office_cost','other_cost','depreciation','interest','principal']]
result=result.fillna(0)
def money_trim(array):
    result=round(array,-1)
    result=result.astype(int)
    return result
result=result.apply(money_trim)
result

#쿼터로 만들기
result_quarter=result.copy()
result_quarter.index=result_quarter.index.to_period("q")
result_quarter=result_quarter.groupby(result_quarter.index).sum()
result_quarter.index=pd.date_range(start=result_quarter.index.to_timestamp()[0],periods=result_quarter.shape[0],freq="q")

#연단위 로 만들기
result_year=result.copy()
result_year.index=result_year.index.to_period("y")
result_year=result_year.groupby(result_year.index).sum()
result_year.index=pd.date_range(start=result_year.index.to_timestamp()[0],periods=result_year.shape[0],freq="y")

#손익계산서 만들기 
def makeIncomeState(income_state):
    income_state["gross_income"]=income_state["smp_revenue"]+income_state["rec_revenue"]
    income_state["operating_expense"]=income_state["monitoring_cost"]+income_state["elec_safety_cost"]+income_state["office_cost"]+income_state["other_cost"]+income_state["depreciation"]                                                                      
    income_state["operating_income"]=income_state["gross_income"]-income_state["operating_expense"]
    income_state["pretax_net_income"]=income_state["operating_income"]-income_state["interest"]
    def tax(x):
        if x>0:
            return x*0.1
        else:
            return 0
    income_state["tax"]=income_state.pretax_net_income.apply(tax)
    income_state["net_income"]=income_state["pretax_net_income"]-income_state["tax"]
    income_state=income_state.loc[:,["smp_revenue","rec_revenue","gross_income","monitoring_cost","elec_safety_cost","office_cost","other_cost","depreciation","operating_expense","operating_income","interest","pretax_net_income","tax","net_income"]]
    return income_state

income_month=makeIncomeState(result.copy())
income_quarter=makeIncomeState(result_quarter.copy())
income_year=makeIncomeState(result_year.copy())

/Users/youngji/.local/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,smp_revenue,rec_revenue,OM_cost,monitoring_cost,elec_safety_cost,office_cost,other_cost,depreciation,interest,principal
2018-07-31,0,0,0,0,0,0,0,0,0,0
2018-08-31,0,0,0,0,0,0,0,0,502530,607340
2018-09-30,0,0,0,0,0,0,0,0,502530,607340
2018-10-31,596760,1065640,1243750,104000,100000,1041670,22880,1041670,502530,607340
2018-11-30,846830,1512200,1244790,104090,100080,1042530,22900,1041670,502530,607340
2018-12-31,858840,1552120,1245820,104170,100170,1043400,22920,1041670,502530,607340
2019-01-31,853040,1541630,1246860,104260,100250,1044270,22940,1041670,502530,607340
2019-02-28,765240,1382970,1247900,104350,100330,1045140,22960,1041670,502530,607340
2019-03-31,841430,1520660,1248930,104430,100420,1046010,22980,1041670,502530,607340
2019-04-30,798930,1461460,1249970,104520,100500,1046870,23000,1041670,502530,607340


In [ ]:
cash_flow=result.copy()
cash_flow=pd.concat([cash_flow,income_year.tax],axis=1)
cash_flow=cash_flow.fillna(0)
cash_flow["operation_cash_in"]=cash_flow["smp_revenue"]+cash_flow["rec_revenue"];
cash_flow["operation_cash_out"]=cash_flow["monitoring_cost"]+cash_flow["elec_safety_cost"]+cash_flow["office_cost"]+cash_flow["other_cost"]+cash_flow["interest"]+cash_flow["tax"]
cash_flow["operation_cash"]=cash_flow["operation_cash_in"]-cash_flow["operation_cash_out"]

cash_flow["finance_cash_in"]=0
cash_flow["acqusition_asset"]=0;cash_flow["acqusition_asset"][0]=params["construction"]
cash_flow["finance_cash_out"]=cash_flow["acqusition_asset"]
cash_flow["finance_cash"]=cash_flow["finance_cash_in"]-cash_flow["finance_cash_out"]

cash_flow["debt"]=0; cash_flow["debt"][0]=params["principal"]
cash_flow["equity"]=0; cash_flow["equity"][0]=params["equity"]
cash_flow["investment_cash_in"]=cash_flow["debt"]+cash_flow["equity"]
cash_flow["investment_cash_out"]=cash_flow["principal"]
cash_flow["investment_cash"]=cash_flow["investment_cash_in"]-cash_flow["investment_cash_out"]

#현금흐름표 만들기 
cash_flow["start_cash"]=0;cash_flow["end_cash"]=0;
cash_flow["cash_change"]=cash_flow["operation_cash"]+cash_flow["finance_cash"]+cash_flow["investment_cash"]
for i in range(0,cash_flow.shape[0]-1):
    cash_flow["end_cash"][i]=cash_flow["start_cash"][i]+cash_flow["cash_change"][i];
    cash_flow["start_cash"][i+1]=cash_flow["end_cash"][i];
cash_flow["end_cash"][cash_flow.shape[0]]=cash_flow["start_cash"][cash_flow.shape[0]]+cash_flow["cash_change"][cash_flow.shape[0]]
cash_flow

,smp_revenue,rec_revenue,gross_income,monitoring_cost,elec_safety_cost,office_cost,other_cost,depreciation,operating_expense,operating_income,interest,pretax_net_income,tax,net_income
2018Q3,0,0,0,0,0,0,0,0,0,0,1005060,-1005060,0,-1005060
2018Q4,2302430,4129960,6432390,312260,300250,3127600,68700,3125010,6933820,-501430,1507590,-2009020,0,-2009020
2019Q1,2459710,4445260,6904970,313040,301000,3135420,68880,3125010,6943350,-38380,1507590,-1545970,0,-1545970
2019Q2,2426190,4402300,6828490,313820,301750,3143220,69060,3125010,6952860,-124370,1507590,-1631960,0,-1631960
2019Q3,2449680,4357640,6807320,314600,302500,3151050,69230,3125010,6962390,-155070,1507590,-1662660,0,-1662660
2019Q4,2436440,4266420,6702860,315390,303250,3158880,69390,3125010,6971920,-269060,1507590,-1776650,0,-1776650
2020Q1,2398350,4135070,6533420,316170,304010,3166770,69570,3125010,6981530,-448110,1507590,-1955700,0,-1955700
2020Q2,2357900,4050110,6408010,316960,304760,3174660,69750,3125010,6991140,-583130,1507590,-2090720,0,-2090720
2020Q3,2351970,4009020,6360990,317750,305530,3182550,69930,3125010,7000770,-639780,1507590,-2147370,0,-2147370
2020Q4,2302730,3925110,6227840,318540,306290,3190470,70080,3125010,7010390,-782550,1507590,-2290140,0,-2290140
